# Tail calls and tail recursion

*Unlike some languages, Python does not treat tail calls differently from other function calls. But the concept is still meaningful and useful.*

### Basic definition

A function call `f(...)` whose result is immediately returned is a *tail call*:

```python
return f(...)
```

If any operation, no matter how trivial, ever occurs between receiving the result and returning it, then the call is not a tail call. So the call to `f` is not a tail call here:

```python
return 1 + f(...)  # The last operation was +, not the call to f.
```

Likewise, even the call to `f` in `return f(...)` is not a tail call when that statement appears in:

- a `try` block that has an associated `finally` block, *or*
- a `with` statement.

***Learning check 1:*** Why does that prevent it from being a tail call?

### In (conceptually) void functions

When a void function calls another void function and then immediately returns, that is also a tail call. Python doesn't have void functions. But if `f` never explicitly returns a value, then `f(...)` always evaluates to `None`, so the function call in

```python
f(...)
return  # Or an implicit return by falling off the end.
```

can be regarded as a tail call, in that it always has the same effect as:

```python
return f(...)
```

However, this differs from other tail calls in that its status as such depends on knowledge of the called function's behavior, since Python is dynamically typed. Outside of this special "void" case, it is possible to infer only from looking at the call site itself whether a call is a tail call. (Though one must still always check that the call site isn't inside a `try` block with an associated `finally` block and isn't inside a `with` statement.)

### Tail recursion (definition)

A *tail-recursive* function is a recursive function whose recursive calls are all tail calls.

### Proper tail calls

A language is said to have *proper tail calls* if, whenever a tail call is made, the caller's stack frame is replaced by the callee's stack frame, so that when $A$ makes a non-tail call to $B$, and $B$ makes a tail call to $C$, $C$ returns directly to $A$. Likewise, if $A$ makes a non-tail call to $B$, and $B$ makes a tail call to $C$, which makes a tail call to $D$, which makes a tail call to $E$, $E$ returns directly to $A$.

This holds both when some or all of the functions are the same (as happens in tail recursion) and when they are all different. Thus, in languages with proper tail calls, even very "deep" chains of tail calls use only $\mathcal{O}(1)$ call-stack space and don't overflow.

***Learning check 2:*** What is it about tail calls that makes it possible&mdash;in the sense of preserving correct behavior&mdash;for the caller's stack frame to be overwritten, or discarded and replaced, with the callee's stack frame? (This is the most fundamental thing to understand about tail calls&mdash;more so than any definition.)

**Python does *not* have proper tail calls**, so functions like `recursion.semifactorial_tail` and `fibonacci.fibonacci_tail` still raise `RecursionError` for large `n`.

(If a future version of Python were to gain proper tail calls, they still might not apply to the "void" case, since, in Python, that depends on the behavior of the called function.)

### Tail recursion in algorithm design

Even when using languages without proper tail calls, like Python, recognizing tail recursion is nonetheless of practical value. Recognizing tail recursion offers insights into how some algorithms work and presents opportunities for manual optimization. In particular, **tail recursion can be converted to iteration without requiring a stack**. It's obvious that, for example, semifactorials can be computed iteratively; but in less straightforward situations, especially if some recursive calls are tail calls and others aren't, some major optimization opportunities can be hard to identify without tail recursion in mind.

When a unary function that is natural to implement recursively with no helper function is implemented tail-recursively, it is often necessary to use a helper function. The helper function's additional parameters correspond to the non-parameter variables an iterative implementation would introduce to maintain state across iterations.

For example, consider summing the elements of a singly linked list with the iterative accumulator pattern:

```python
def sll_sum_iter(head):
    acc = 0
    while head:
        acc += head.value
        head = head.next
    return acc
```

Implemented tail-recursively, the accumulator pattern looks like:

```python
def sll_sum_tailrec(head):
    def helper(acc, node):
        return helper(acc + node.value, node.next) if node else acc

    return accumulate(0, head)
```

(Here, the helper function doesn't capture any variables, and therefore could just as reasonably be implemented as a top-level nonpublic function, provided it were given a more descriptive name.)

Contrast this to the *non-tail* recursive strategy:
    
```python
def sll_sum_rec(head):
    return head.value + sll_sum_rec(head.next) if head else 0
```

***Learning check 3:*** In `sll_sum_tailrec`, `helper` is tail recursive: its call to itself is a tail call. But how can that be? The operand of `return` is not a function call to `helper`; instead, it is a larger expression that contains a function call to `helper`.

Recall that, in `return f(...)`, the call to `f` is *usually* a tail call (when is it not?), but that even in `return 1 + f(...)`, the call to `f` is *never* a tail call. The expression returned by `helper` is even more complicated than that! So how can the recursive call to `helper` be a tail call? (Note that this is *not* like the "void" case. That the call to `helper` is a tail call can still be inferred just by looking at the call site.)

***Learning check 4:*** Of all the recursive functions already implemented in `recursion.py`, which are tail recursive and which are not?

***Learning check 5:*** In one of the algorithms implemented recursively and then iteratively in `recursion.py`, reasoning about tail calls would have sped up your translation from recursive to iterative, and also supplied greater confidence in the iterative implementation's correctness. (It's not that you would've written different code; your recursive implementation was already tail recursive.)

Which functions were these? Paste their implementations, with docstrings removed or shortened to single lines), below. Annotate the corresponidng parts of each with comments, showing:

- How one or more function parameters in the recursive implementation became local variables in the iterative implementation.

- How each return in the recursive implementation turned into code that updated the value of at least one such local variable. (One way to show this correspondence is to mark the corresponding code with short labels like `# A`, `# B`, `# C`, etc.)

### Note on external exercises

The fundamental concepts of tail calls and tail recursion have been covered above.

If you like, you can do any exercises related to tail recursion in `recursion.py` and `fibonacci.py` before reading the rest of this writeup.

### Mutual tail recursion

As you may recall, in *mutual recursion*, a function `f` calls a different function `g`, which in turn calls `f`. Mutual recursion usually involves exactly two functions calling each other (but the recursion depth may be far greater than $2$). Occasionally, a relationship of mutual recursion exists between three or more functions.

*Mutual tail recursion*&mdash;mutual recursion where the indirect recursive calls are all tail calls&mdash;is possible. Here's a contrived but illuminating example:

```python
def is_even(n):
    return n == 0 or is_odd(n - 1)
```

```python
def is_odd(n):
    return n != 0 and is_even(n - 1)
```

The top-level caller is responsible for ensuring the original argument is a nonnegative `int`. (This example is contrived because determining the parity of a number can be done in constant time, yet those functions take time linear in the value of their argument.)

***Learning check 6:*** Those functions are mutually tail recursive: the call in `is_even` to `is_odd` is a tail call, the call in `is_odd` to `is_even` is a tail call, and there are no other recursive calls. But how can that be? The operands of `return` in `is_even` and `is_odd` are not function calls to `is_odd` and `is_even`, respectively. Instead, they are larger expressions that contain functon calls to `is_odd` and `is_even`. So how are the calls contained in them tail calls?

*Note:* To know the functions are mutually tail recursive, you would have to look at both, but that's not the question here&mdash;this is just asking why the calls are tail calls. To know that `is_even`'s call to `is_odd` is a tail call (and that `is_odd`'s call to `is_even` is a tail call), you don't need to look inside the function being called. So this, too, is *not* like the "void" case&mdash;that each is a tail call can still be inferred just by looking at each call site. Thus, you should ensure your explanation makes no use of your knowledge that `is_even` and `is_odd` return `bool`. For example, if `is_odd` were monkey-patched to refer to a different function with a different return type, the call to `is_odd` in `is_even` would still be a tail call.

***Learning check 7:*** Recall how a recursive function that is memoized by `@functools.cache` or your `@memoize` or `@memoize_by` decorators is actually mutually recursive. Explain: why that is, what the two separate functions are that call each other, and which of them is assigned to the name you define your function with. Then consider: Is it possible for mutual recursion that arises from decorating a recursive definition with one of those decorators to be mutual *tail* recursion? If so, give an example (either by describing it in prose or by writing code). If not, explain why this can never happen.

### Tail calls and other language features

You've considered how `try`-`finally` and `with` limit what can be a tail call. Let's look a little deeper into the issue of how *cleanup* affects how tail calls can be treated.

Recall that it is possible for the destruction of an object to have a side effect. For example, a generator function can have a `yield` statement in a `try` block with a corresponding `finally` block. If the function is called, and `next` is called on the returned generator object, and the `yield` statement in the `try` block is reached, *and then the generator object is destroyed*, this will cause the `finally` block to run. (Otherwise, it would rarely be reasonable to, for example, have a `yield` statement inside a `with` statement.)

A Python implementation is permitted to make stronger guarantees than the language itself. It's worth considering how guarantees about tail calls would interact with guarantees about garbage collection. CPython uses a hybrid garbage collection strategy that includes reference counting. But the description given below is simplified. This hypothetical may shed light on one reason Python not having proper tail calls is a defensible design choice.

***Learning check 8:*** Suppose a Python implementation guarantees that it uses reference-counting garbage collection. Each object has a reference count, and when the reference count is decremented to zero, the object is collected. If a local variable refers to an object, and the variable is bound to a new object or deleted, the object it referred to has its reference count decremented. When execution permanently leaves the scope of a local variable, it is as though the variable is deleted, unless its lifetime is prolonged by being captured.

Suppose that, then, a new version of the Python language gets proper tail calls (i.e., as part of the semantics of function calls, tail calls are required to replace the caller's stack frame). So the next release of this implementation implements proper tail calls. It still uses reference-counting garbage collection. But programmers' expectations about the effects of that garbage-collection scheme may now mislead them. Why is that, and what should the implementation's documentation warn programmers about?

*Note:* Although some generator functions contain code that might be regarded as tail calls (a `yield from` followed immediately by returning), this question is not about that. You need consider only the effect of tail calls in non-generator functions to answer this.